In [1]:
import os
import time
import copy
import torch
import torch.optim as optim
# 引入线性回归常用的损失函数
from torch.nn import Linear, Module, MSELoss
import pprint as pp
import utils.hypergraph_utils as hgut
from models import HGNN
from config import get_config
from datasets import load_feature_construct_H
from datasets import load_ft
import pandas as pd
import numpy as np

In [2]:
# initialize data
data_dir = r"L:\HGNN try1\CG_dataset_clsf.mat"
fts, lbls, idx_train, idx_test = load_ft(data_dir,'CGNN')

In [3]:
print(fts, lbls, idx_train, idx_test)

[[1.5000e+01 3.6440e+03 0.0000e+00 ... 3.3100e+02 4.0500e+02 9.5000e+01]
 [1.0000e+00 3.4800e+02 0.0000e+00 ... 4.3000e+01 3.2000e+01 1.5000e+01]
 [1.0000e+00 5.5100e+02 0.0000e+00 ... 5.8000e+01 3.7000e+01 4.0000e+00]
 ...
 [8.0000e+00 6.9700e+02 3.0000e+00 ... 4.7000e+01 6.5000e+01 2.2000e+01]
 [8.0000e+00 9.5000e+02 0.0000e+00 ... 7.8000e+01 1.2200e+02 3.2000e+01]
 [1.8000e+01 1.9562e+04 0.0000e+00 ... 1.9200e+03 1.6200e+03 4.0000e+02]] [[3 3 3 ... 3 3 3]] [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105

In [4]:
# H_dir
H_dir = r"L:\hypergraph prediction\话题用户超图.csv"
W_dir = r"L:\hypergraph prediction\用户超边权值.csv"
H = pd.read_csv(H_dir,header = None)
W = pd.read_csv(W_dir)
H = H.to_numpy()
W = W.iloc[:,1].to_numpy()

In [5]:
G = hgut.generate_G_from_H(H,W, variable_weight=False)

L:\HGNN try1\utils\hypergraph_utils.py:115: RuntimeWarning: divide by zero encountered in power
  DV2 = np.mat(np.diag(np.power(DV, -0.5)))


In [6]:
G = torch.Tensor(G)

In [7]:
G = torch.where(torch.isnan(G), torch.full_like(G, 0), G)
G

tensor([[0.1177, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0435]])

In [8]:
'''
fts, lbls, idx_train, idx_test, H = \
    load_feature_construct_H(data_dir,
                             m_prob=cfg['m_prob'],
                             K_neigs=cfg['K_neigs'],
                             is_probH=cfg['is_probH'],
                             use_mvcnn_feature=cfg['use_mvcnn_feature'],
                             use_gvcnn_feature=cfg['use_gvcnn_feature'],
                             use_mvcnn_feature_for_structure=cfg['use_mvcnn_feature_for_structure'],
                             use_gvcnn_feature_for_structure=cfg['use_gvcnn_feature_for_structure'])
'''

# n_class = int(lbls.max()) + 1
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # 指定存放的位置 cpu/gpu

# transform data to device
fts= torch.Tensor(fts).to(device)
lbls = torch.Tensor(lbls).squeeze().long().to(device)
G = torch.Tensor(G).to(device)
idx_train = torch.Tensor(idx_train).long().to(device)
idx_test = torch.Tensor(idx_test).long().to(device)

In [23]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, print_freq=500):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    # 分类任务的评估指标acc
    best_acc = 0.0
#     best_loss = 555


    for epoch in range(num_epochs):
        # 间隔输出训练效果
        if epoch % print_freq == 0:
            print('-' * 10)
            print(f'Epoch {epoch}/{num_epochs - 1}')

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']: # 每一轮都训一遍再测一遍
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            idx = idx_train if phase == 'train' else idx_test # 选择相应数据集

            # Iterate over data.
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(fts, G) # model 输出的是n*1
#                 print(outputs)
                loss = criterion(outputs[idx], lbls[idx]) # 根据自定义的criterion计算训练误差
                _, preds = torch.max(outputs, 1)

                # backward + optimize only if in training phase 在训练阶段进行反向传播和优化
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * fts.size(0)
#             print(fts.size(0))
            running_corrects += torch.sum(preds[idx] == lbls.data[idx])

            epoch_loss = running_loss / len(idx)
            epoch_acc = running_corrects.double() / len(idx)

            if epoch % print_freq == 0:
                print(f'{phase} Loss: {epoch_loss:.4f} Acc:{epoch_acc:4f}')

            # deep copy the model 存储预测效果最好时候的model
            # deep copy 复制完整对象并且存储在独立的内存地址
            # 在评估阶段选择最好的模型，loss最小
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        if epoch % print_freq == 0:
            print(f'Best val acc:{best_acc:4f}')
            print('-' * 20)

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val acc:{best_acc:4f} ')

    # load best model weights
    model.load_state_dict(best_model_wts) # 读出最好的model并且返回
    return model

In [24]:
def _main():
    '''
    print(f"Classification on {cfg['on_dataset']} dataset!!! class number: {n_class}")
    print(f"use MVCNN feature: {cfg['use_mvcnn_feature']}")
    print(f"use GVCNN feature: {cfg['use_gvcnn_feature']}")
    print(f"use MVCNN feature for structure: {cfg['use_mvcnn_feature_for_structure']}")
    print(f"use GVCNN feature for structure: {cfg['use_gvcnn_feature_for_structure']}")
    print('Configuration -> Start')
    pp.pprint(cfg)
    print('Configuration -> End')
    '''

    # 定义模型，此处分类模型被耦合在HGNN函数中，需要去HGNN那里改预测任务
    model_ft = HGNN(in_ch=fts.shape[1], # 特征矩阵列数
                    n_class=4,
                    n_hid=128,
                    dropout=0)
    model_ft = model_ft.to(device) # 将模型转移到设备上

    # adam优化
    optimizer = optim.Adam(model_ft.parameters(), lr=0.01,weight_decay=0.005)
    # SGD优化
#     optimizer = optim.SGD(model_ft.parameters(), lr=0.01, weight_decay=cfg['weight_decay)'])

    # 动态调整学习率
    # milestone 调整轮次
    # gamma 调整倍数
    # new_lr = lr*gamma when epoch = milestone[i]
    schedular = optim.lr_scheduler.MultiStepLR(optimizer,
                                               milestones=[100,300,500,800],
                                               gamma=0.9)
    criterion = torch.nn.CrossEntropyLoss() # 分类预测任务中使用了交叉熵损失函数
    # 线性回归要改成
#     criterion = torch.nn.MSELoss(reduction = 'mean')

    model_ft = train_model(model_ft, criterion, optimizer, schedular, 1500, print_freq=100)

In [25]:
if __name__ == '__main__':
    _main()

----------
Epoch 0/1499
train Loss: 8.1433 Acc:0.766706
val Loss: 1595.7900 Acc:0.028169
Best val acc:0.028169
--------------------
----------
Epoch 100/1499
train Loss: 0.6419 Acc:0.889801
val Loss: 1.8919 Acc:0.943662
Best val acc:0.943662
--------------------
----------
Epoch 200/1499
train Loss: 0.5168 Acc:0.896835
val Loss: 1.6489 Acc:0.943662
Best val acc:0.943662
--------------------
----------
Epoch 300/1499
train Loss: 0.5266 Acc:0.890973
val Loss: 1.5446 Acc:0.943662
Best val acc:0.943662
--------------------
----------
Epoch 400/1499
train Loss: 0.4579 Acc:0.893318
val Loss: 1.4001 Acc:0.943662
Best val acc:0.948357
--------------------
----------
Epoch 500/1499
train Loss: 0.4334 Acc:0.908558
val Loss: 1.4167 Acc:0.943662
Best val acc:0.948357
--------------------
----------
Epoch 600/1499
train Loss: 0.4236 Acc:0.907386
val Loss: 1.4092 Acc:0.938967
Best val acc:0.948357
--------------------
----------
Epoch 700/1499
train Loss: 0.4181 Acc:0.910903
val Loss: 1.3775 Acc:0.9

In [ ]:
import utils.hypergraph_utils as hgut

In [179]:
G = hgut.generate_G_from_H(H, W, variable_weight=False)
G= torch.Tensor(G)
G = torch.where(
    torch.isnan(G),
    torch.full_like(G, 0),
    G)

L:\HGNN try1\utils\hypergraph_utils.py:115: RuntimeWarning: divide by zero encountered in power
  DV2 = np.mat(np.diag(np.power(DV, -0.5)))


In [196]:
import os
import time
import copy
import torch
import torch.optim as optim
# 引入线性回归常用的损失函数
from torch.nn import Linear, Module, MSELoss
import pprint as pp
import utils.hypergraph_utils as hgut
from models import HGNN
from config import get_config
import pandas as pd
import numpy as np

In [209]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # 指定存放的位置 cpu/gpu
data_dir = r"L:\HGNN try1\CG_dataset.mat"
fts, lbls, idx_train, idx_test = load_ft(data_dir,'CGNN')
# transform data to device
fts = torch.Tensor(fts).to(device)
lbls = torch.Tensor(lbls).squeeze().long().to(device)
G = torch.Tensor(G).to(device)
idx_train = torch.Tensor(idx_train).long().to(device)
idx_test = torch.Tensor(idx_test).long().to(device)

In [210]:
fts = torch.nn.functional.normalize(fts, p=1, dim=1)

In [211]:
print(fts,
      lbls,
      G,
      idx_train,
      idx_test)

tensor([[3.0426e-03, 7.3915e-01, 0.0000e+00,  ..., 6.7140e-02, 8.2150e-02,
         1.9270e-02],
        [7.7519e-04, 2.6977e-01, 0.0000e+00,  ..., 3.3333e-02, 2.4806e-02,
         1.1628e-02],
        [1.1534e-03, 6.3552e-01, 0.0000e+00,  ..., 6.6897e-02, 4.2676e-02,
         4.6136e-03],
        ...,
        [4.8485e-03, 4.2242e-01, 1.8182e-03,  ..., 2.8485e-02, 3.9394e-02,
         1.3333e-02],
        [6.2647e-03, 7.4393e-01, 0.0000e+00,  ..., 6.1081e-02, 9.5536e-02,
         2.5059e-02],
        [7.6120e-04, 8.2725e-01, 0.0000e+00,  ..., 8.1194e-02, 6.8508e-02,
         1.6915e-02]]) tensor([3, 3, 3,  ..., 3, 3, 3]) tensor([[0.1177, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.2500,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ...,

In [ ]:
'''
n_class = int(lbls.max()) + 1
print(n_class)
model_ft = HGNN(in_ch=fts.shape[1], # 特征矩阵列数
                    n_class=n_class,
                    n_hid=128,
                    dropout=0.5)
optimizer = optim.Adam(model_ft.parameters(), lr=0.001,
                           weight_decay=0.0005) # adam优化
    # optimizer = optim.SGD(model_ft.parameters(), lr=0.01, weight_decay=cfg['weight_decay)
schedular = optim.lr_scheduler.MultiStepLR(optimizer,
                                               milestones=[100],
                                               gamma=0.9)
criterion = torch.nn.CrossEntropyLoss() # 分类预测任务中使用了交叉熵损失函数
outputs = model_ft(fts, G)
print(outputs)
loss = criterion(outputs[idx_train], lbls[idx_train]) # 根据自定义的criterion计算训练误差
_, preds = torch.max(outputs, 1)# torch.max(input,dim) 在行上返回每行的最大值，并返回索引
print(preds)

MSE = torch.nn.MSELoss(reduction = 'mean')
loss = MSE(preds.float(),lbls.float())
print(loss)
'''

In [261]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, print_freq=500):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        if epoch % print_freq == 0:
            print('-' * 10)
            print(f'Epoch {epoch}/{num_epochs - 1}')

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()  # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            idx = idx_train if phase == 'train' else idx_test # 选择相应数据集

            # Iterate over data.
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(fts, G)
                loss = criterion(outputs[idx], lbls[idx]) # 根据自定义的criterion计算训练误差
                _, preds = torch.max(outputs, 1)

                # backward + optimize only if in training phase 在训练阶段进行反向传播和优化
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            # statistics
            running_loss += loss.item() * fts.size(0)
            running_corrects += torch.sum(preds[idx] == lbls.data[idx])

            epoch_loss = running_loss / len(idx)
            epoch_acc = running_corrects.double() / len(idx)

            if epoch % print_freq == 0:
                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model 存储预测效果最好时候的model
            # deep copy 复制完整对象并且存储在独立的内存地址
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        if epoch % print_freq == 0:
            print(f'Best val Acc: {best_acc:4f}')
            print('-' * 20)

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts) # 读出最好的model并且返回
    return model

In [262]:
def _main():
 
    n_class = int(lbls.max()) + 1
    # 定义模型，此处分类模型被耦合在HGNN函数中，需要去HGNN那里改预测任务
    model_ft = HGNN(in_ch=fts.shape[1], # 特征矩阵列数
                    n_class=n_class,
                    n_hid=128,
                    dropout=0.1)
    model_ft = model_ft.to(device) # 将模型转移到设备上

    optimizer = optim.Adam(model_ft.parameters(), lr=0.001,
                           weight_decay=0.0005) # adam优化
    # optimizer = optim.SGD(model_ft.parameters(), lr=0.01, weight_decay=cfg['weight_decay)
    schedular = optim.lr_scheduler.MultiStepLR(optimizer,
                                               milestones=[100],
                                               gamma=0.9)
    criterion = torch.nn.CrossEntropyLoss() # 分类预测任务中使用了交叉熵损失函数

    model_ft = train_model(model_ft, criterion, optimizer, schedular,1000, print_freq=100)

In [263]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# cfg = get_config('config/config.yaml')


if __name__ == '__main__':
    _main()

----------
Epoch 0/999
train Loss: 1.7092 Acc: 0.0445
val Loss: 6.7558 Acc: 0.0188
Best val Acc: 0.018779
--------------------
----------
Epoch 100/999
train Loss: 1.0054 Acc: 0.7644
val Loss: 3.2953 Acc: 0.7934
Best val Acc: 0.793427
--------------------
----------
Epoch 200/999
train Loss: 0.9896 Acc: 0.7644
val Loss: 3.2022 Acc: 0.7934
Best val Acc: 0.793427
--------------------
----------
Epoch 300/999
train Loss: 0.9847 Acc: 0.7644
val Loss: 3.1948 Acc: 0.7934
Best val Acc: 0.793427
--------------------
----------
Epoch 400/999
train Loss: 0.9809 Acc: 0.7644
val Loss: 3.2040 Acc: 0.7934
Best val Acc: 0.793427
--------------------
----------
Epoch 500/999
train Loss: 0.9791 Acc: 0.7644
val Loss: 3.1971 Acc: 0.7934
Best val Acc: 0.793427
--------------------
----------
Epoch 600/999
train Loss: 0.9755 Acc: 0.7644
val Loss: 3.1877 Acc: 0.7934
Best val Acc: 0.793427
--------------------
----------
Epoch 700/999
train Loss: 0.9707 Acc: 0.7644
val Loss: 3.1821 Acc: 0.7934
Best val Acc: 

In [236]:
n_class = int(lbls.max()) + 1
model_try = HGNN(in_ch=fts.shape[1], # 特征矩阵列数
                    n_class=n_class,
                    n_hid=128,
                    dropout=0.5)

In [237]:
model_try(fts,G)

tensor([[-0.3880, -0.0823, -0.1223, -0.1648],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.4839, -0.0824, -0.1590, -0.2594],
        ...,
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.5691, -0.0444, -0.1979, -0.2617]], grad_fn=<MmBackward0>)